In [1]:
import polars as pl

In [212]:
dfs = (
    pl.read_csv("../src/audit_reports/*.csv", infer_schema_length=0)
    .with_columns(
        pl.col("Report Type").str.split(" ").list.get(-1).alias("Report_Year"),
        pl.col("Date Published")
        .str.split(" ")
        .list.get(-1)
        .str.to_date("%m/%d/%Y")
        .alias("Report_Published_Date"),
        pl.col("Report Type")
        .str.split(" ")
        .list.slice(0, 2)
        .list.join(" ")
        .alias("Report_Category"),
        pl.col("Latest Submission").str.to_date("%m/%d/%y"),
    )
    .drop("TOTAL")
    .filter(~pl.col("PFI").str.contains("TOTAL"))
)

In [222]:
clean_df_piv = (
    (
        dfs.melt(
            id_vars=[
                "PFI",
                "FACILITY",
                "Latest Submission",
                "Report Type",
                "Date Published",
                "Report_Year",
                "Report_Published_Date",
                "Report_Category",
            ],
            variable_name="Report_Month",
            value_name="Records_Submitted",
        )
    )
    .with_columns(pl.col("Records_Submitted").str.replace_all(",", ""))
    .with_columns(pl.col("Records_Submitted").cast((pl.Int32)))
)

In [229]:
clean_df_piv.write_parquet("audit_reports.parquet")